# Application on point cloud

## Load modules 

In [ ]:
import numpy
import pandas

import openalea.phenomenal.display as phm_display
import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display.notebook as phm_display_notebook
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.multi_view_reconstruction as phm_mvr
import openalea.phenomenal.segmentation as phm_seg

## Load cloud point cloud 

First we define a function "read_from_xyz" to load our point cloud data 

In [ ]:
def read_from_xyz(filename):

    xyz_position = list()
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.split()[:3]] # load just position not color
            xyz_position.append(tuple(values))
    f.close()
    
    return numpy.array(xyz_position)

Maize point cloud was found on : https://github.com/CharlieLeee/Maize-plant-point-cloud-dataset and cleaned mannually with CloudCompare software. 3D reconstruction is pretty bad, leaves are very noised and seems cutted.  

In [ ]:
file_path = "maize_point_cloud.pts"
xyz_position = read_from_xyz(file_path)
phm_display_notebook.show_point_cloud(xyz_position, size=0.1)

## Convert point cloud to voxel grid

Once loaded, we attribute for each point a fictive voxel size to simulate the data like a voxel grid. More the fictive voxel size is small more the 3D voxel representation is accurate. After we normalize the pointcloud into a grid.

In [ ]:
fictive_voxel_size = 0.05

voxel_grid = phm_obj.VoxelGrid(numpy.array(xyz_position), fictive_voxel_size)
voxel_grid = phm_obj.VoxelGrid.from_image_3d(
    voxel_grid.to_image_3d(),
    voxels_value=1,
    voxels_size=1, # must be integer
    world_coordinate=(0.0, 0.0, 0.0))

print("Shape Image 3D:", voxel_grid.to_image_3d().shape)

phm_display_notebook.show_voxel_grid(voxel_grid, size=1)

In [ ]:
graph = phm_seg.graph_from_voxel_grid(voxel_grid, connect_all_point=False)
src_node = tuple(max(graph.nodes(), key=lambda d: d[1]))
voxel_skeleton = phm_seg.skeletonize(voxel_grid, graph, src_node=src_node)

## Skeletonization

In [ ]:
phm_display_notebook.show_skeleton(voxel_skeleton, with_voxel=True, size=1.0)

## Maize Segmentation

In [ ]:
vms = phm_seg.maize_segmentation(voxel_skeleton, graph, stem_strategy="longest")

phm_display_notebook.show_segmentation(vms, size=1)

## Maize Analysis

In [ ]:
vmsi = phm_seg.maize_analysis(vms)
phm_display_notebook.show_segmentation(vmsi, size=1)

Take a look, of what kind of data is extract. (pm = phenomenal_mearsurement)

In [ ]:
df = pandas.DataFrame([vo.info for vo in vmsi.voxel_organs]  + [vmsi.info])
df